# **Regresi Ridge**

**Nama : Hilmi Firmansyah** 

Regresi Ridge merupakan salah satu metode yang digunakan untuk mengatasi multikolinearitas data yang dimodifikasi dari metode kuadrat terkecil. Modifikasi dilakukan dengan penambahan tetapan bias/penalized/penalty λ pada diagonal matriks yang memengaruhi besarnya koefisien penduga ridge.

Perhatikan bahwa untuk Least Square Estimation dengan persamaan 
\begin{equation}
y_i=\beta_0+\beta_1 x_{1i} +\beta_2 x_{2i}
\end{equation}
maka kita dapat peroleh
\begin{equation}
\epsilon_i= y_i- \left( \beta_0+\beta_1 x_{1i}+\beta_2 x_{2i}\right)
\end{equation}
Dimana
\begin{equation}
\epsilon_i =\begin{pmatrix}  \epsilon_1 \\ \epsilon_2 \\ \vdots \\ \epsilon_n \end{pmatrix} , Y=\begin{pmatrix}  y_1 \\ y_2 \\ \vdots \\ y_n \end{pmatrix}, X=\begin{pmatrix}  1 & x_{11} & x_{21} \\ 1 & x_{12} & x_{22} \\ \vdots & \vdots & \vdots \\ 1 & x_{1n} & x_{2n} \end{pmatrix}, \text{and} \beta_j = \begin{pmatrix}  \beta_{0} \\ \beta_{1} \\ \beta_{2} \end{pmatrix} 
\end{equation}
Lebih lanjut, kita dapat meminimumkan errornya dengan menggunakan derivatif parsial untuk setiap $\beta_j$ dengan $j=0,1,2$ sehingga dapat peroleh persamaan untuk tiap derivatif parsialnya yaitu
\begin{equation}
X^T X \beta_j = X^T Y
\end{equation}
atau dapat diperoleh koefisien tiap $\beta_j$ dengan $j=0,1,2$ yaitu
\begin{equation}
\beta_j = (X^T X)^{-1} X^T Y
\end{equation}
Oleh karena dapat regresi Ridge terdapat penambahan tetapan bias/penalized/penalty λ pada diagonal matriks yang memengaruhi besarnya koefisien penduga ridge, maka untuk penentuan koefisien $\beta_j$ dengan $j=0,1,2$ dapat dituliskan menjadi
\begin{equation}
\beta_j=(X^TX+λI)^{-1}X^Ty
\end{equation}


Pada bahasan di bawah ini, akan dilakukan analisis regresi ridge dengan menggunakan data diabetes yang bersifat logistik dimana Outcome nya akan bernilai 1 jika seseorang terindikasi penyakit diabetes dan bernilai 0 jika seseorang tidak terindikasi penyakit diabetes dengan menggunakan variabel independennya yaitu Age dan Insulin.

Pertama, akan dilakukan import beberapa library yang digunakan dalam analisis regresi ridge dengan menggunakan data diabetes.csv serta menampilkan beberapa baris dan kolom dari data diabetes.csv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
diabetes = pd.read_csv("diabetes.csv")

In [3]:
diabetes

,Unnamed: 0,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0,1,148.0,72.0,35.000000,79.799479,33.6,0.627,50,1
1,1,1,85.0,66.0,29.000000,79.799479,26.6,0.351,31,0
2,2,1,183.0,64.0,20.536458,79.799479,23.3,0.672,32,1
3,3,1,89.0,66.0,23.000000,94.000000,28.1,0.167,21,0
4,4,0,137.0,40.0,35.000000,168.000000,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...,...
763,763,1,101.0,76.0,48.000000,180.000000,32.9,0.171,63,0
764,764,1,122.0,70.0,27.000000,79.799479,36.8,0.340,27,0
765,765,1,121.0,72.0,23.000000,112.000000,26.2,0.245,30,0
766,766,1,126.0,60.0,20.536458,79.799479,30.1,0.349,47,1


Kemudian, lakukan train data dan test untuk data diabetes yang dimiliki dengan variabel targetnya adalah "Outcome" serta variabel prediktornya adalah "Age" dan "Insulin". Lebih lanjut, dimisalkan X merupakan data train dari prediktor dan Y merupakan data train dari target.

In [ ]:
train, test = train_test_split(diabetes, test_size=0.2, random_state=1)

In [ ]:
predictors = ["Age", "Insulin"]
target = "Outcome"

In [ ]:
X = train[predictors].copy()
y = train[[target]].copy()

In [ ]:
X

,Age,Insulin
663,40,130.000000
712,38,79.799479
161,45,105.000000
509,64,79.799479
305,29,105.000000
...,...,...
645,30,440.000000
715,34,392.000000
72,42,79.799479
235,26,79.799479


In [ ]:
y

,Outcome
663,1
712,1
161,0
509,0
305,0
...,...
645,0
715,1
72,1
235,1


Berdasarkan variabel X yang telah diperoleh di atas, akan dicari rata-rata/mean dari X serta standar deviasi dari X yang kemudian ditampilkan hasilnya masing-masing.

In [ ]:
x_mean = X.mean()
x_std = X.std()

In [ ]:
x_mean

Age         33.298046
Insulin    118.805604
dtype: float64

In [ ]:
x_std

Age        11.810249
Insulin    94.286962
dtype: float64

Selanjutnya, dari data X yang telah dimiliki, dilakukan scaling data agar data yang dimiliki lebih kecil sehingga membuat perhitungan regresi menjadi lebih sederhana. Scaling data X dapat dilakukan dengan formula

$X=\frac{X-\text{x_mean}}{\text{x_std}}$

In [ ]:
X = (X - x_mean) / x_std

In [ ]:
X

,Age,Insulin
663,0.567469,0.118727
712,0.398125,-0.413696
161,0.990830,-0.146421
509,2.599603,-0.413696
305,-0.363925,-0.146421
...,...,...
645,-0.279253,3.406562
715,0.059436,2.897478
72,0.736814,-0.413696
235,-0.617942,-0.413696


Didefinisikan X sebagai matriks yang berisikan kolom terdiri dari kolom intercept serta prediktor. Kemudian, dilakukan pendeskripsian nilai X tersebut dan menampilkannya dimana terdiri berbagai macam informasi seperti count, mean, std, dan lain sebagainya.

In [ ]:
X["intercept"] = 1

In [ ]:
X = X[["intercept"] + predictors]

In [ ]:
X

,intercept,Age,Insulin
663,1,0.567469,0.118727
712,1,0.398125,-0.413696
161,1,0.990830,-0.146421
509,1,2.599603,-0.413696
305,1,-0.363925,-0.146421
...,...,...,...
645,1,-0.279253,3.406562
715,1,0.059436,2.897478
72,1,0.736814,-0.413696
235,1,-0.617942,-0.413696


In [ ]:
X.describe()

,intercept,Age,Insulin
count,614.0,6.140000e+02,6.140000e+02
mean,1.0,2.256610e-16,9.257886e-17
std,0.0,1.000000e+00,1.000000e+00
min,1.0,-1.041303e+00,-1.100954e+00
25%,1.0,-7.872861e-01,-4.136958e-01
50%,1.0,-3.639251e-01,-4.136958e-01
75%,1.0,6.521416e-01,6.569727e-02
max,1.0,3.276980e+00,7.712566e+00


Bagian ini, ditampilkan transpose dari matriks X guna untuk perhitungan selanjutnya

In [ ]:
X.T

,663,712,161,509,305,426,13,480,682,607,...,508,749,583,129,144,645,715,72,235,37
intercept,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Age,0.567469,0.398125,0.990830,2.599603,-0.363925,-0.702614,2.176241,0.144108,-0.956631,-0.702614,...,-1.041303,1.414192,0.736814,2.430258,-0.871958,-0.279253,0.059436,0.736814,-0.617942,1.075503
Insulin,0.118727,-0.413696,-0.146421,-0.413696,-0.146421,-0.413696,7.712566,2.218699,-0.146421,-0.825200,...,-0.453993,-0.413696,-0.413696,-0.413696,1.752039,3.406562,2.897478,-0.413696,-0.413696,-0.413696


Selanjutnya, didefinisikan alpha=2 sebagai penalized/penalty yang akan digunakan dalam perhitungan regresi rigde serta matriks identitas dengan ukuran menyesuaikan dari ukuran matriks X. Kemudian, dilakukan perhitungan penalty yang diperoleh dari perkalian alpha dan matriks Identitas serta ditampilkan hasilnya

In [ ]:
alpha = 2
I = np.identity(X.shape[1])
penalty = alpha * I

In [ ]:
penalty

array([[2., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

Kemudian, dari matriks penalty yang telah diperoleh, untuk baris 1 kolom 1 nilainya diubah menjadi 0 karena dalam perhitungan perkalian di bawah untuk intercept y tidak dilakukan penalized/penalty sehingga baris 1 bernilai 0 untuk setiap kolom.

In [ ]:
penalty[0][0] = 0

In [ ]:
penalty

array([[0., 0., 0.],
       [0., 2., 0.],
       [0., 0., 2.]])

Dengan demikian, dapat diperoleh koefisien-koefisien dari regresi ridge dengan formula

$β=(X^TX+λI)^{-1}X^Ty$

In [ ]:
B = np.linalg.inv(X.T @ X + penalty) @ X.T @ y

In [ ]:
B

,Outcome
0,0.346906
1,0.104871
2,0.088145


Berdasarkan hasil koefiesien-koefisien di atas, maka dapat diperoleh persamaan regresi ridge dimana variabel Y merupakan outcome dari diabetes dan variabel X merupakan variabel independen terdiri dari Age dan Insulin sebagai berikut

$\text{Outcome}=0.346906 + 0.104871 \text{Age} + 0.088145 \text{Insulin}$